# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 24 нояб

__Жесткий дедлайн: 27 нояб


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

In [2]:
msg = emails.sample().iloc[0]['message']
print(msg)

Message-ID: <3583016.1075847550228.JavaMail.evans@thyme>
Date: Fri, 2 Mar 2001 01:31:00 -0800 (PST)
From: tana.jones@enron.com
To: greg.whiting@enron.com, sarah.wesner@enron.com
Subject: (01-78) MARGIN RATE CHANGE FOR HENRY HUB NATURAL GAS FUTURES
 CONTRACTS
Cc: mary.cook@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: mary.cook@enron.com
X-From: Tana Jones
X-To: Greg Whiting, Sarah Wesner
X-cc: Mary Cook
X-bcc: 
X-Folder: \Tanya_Jones_June2001\Notes Folders\Sent
X-Origin: JONES-T
X-FileName: tjones.nsf

----- Forwarded by Tana Jones/HOU/ECT on 03/02/2001 09:31 AM -----

	exchangeinfo@nymex.com
	03/02/2001 08:42 AM
		 
		 To: tana.jones@enron.com
		 cc: 
		 Subject: (01-78) MARGIN RATE CHANGE FOR HENRY HUB NATURAL GAS FUTURES 
CONTRACTS


Notice No. 01-78
March 02, 2001
TO:
ALL NYMEX DIVISION MEMBERS AND MEMBER FIRMS
ALL NYMEX DIVISION CLEARING FIRMS
ALL NYMEX DIVISION OPERATIONS MANAGERS

FROM:
Neal Wolkoff
Executive Vice Pre

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [4]:
import email
import re


def get_body(raw_message):
    b = email.message_from_string(raw_message)
    body = ""

    if b.is_multipart():
        for part in b.walk():
            ctype = part.get_content_type()
            cdispo = str(part.get('Content-Disposition'))

            # skip any text/plain (txt) attachments
            if ctype == 'text/plain' and 'attachment' not in cdispo:
                body = part.get_payload(decode=True)  # decode
                break
    # not multipart - i.e. plain text, no attachments, keeping fingers crossed
    else:
        body = b.get_payload(decode=True)
    
    return body.decode("utf-8")

def clean_email_text(message):
    body = get_body(message)

    # Remove reply headers like 'Re:', 'Cc:', 'Fw:', etc.
    reply_header_pattern = r"^(Re|Fw|Fwd|Cc|Bcc|Subject|To|From|Sent)[^:]*:.*(\n|\r\n)?"
    body = re.sub(reply_header_pattern, '', body, flags=re.MULTILINE | re.IGNORECASE)

    # Remove email addresses, phone numbers, URLs, and filenames
    body = re.sub(r'http\S+|www\S+|https\S+', '', body)
    body = re.sub(r'\b\S+@\S+\b', '', body)
    body = re.sub(r'\b\d{10}\b|\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', '', body)
    # Removes filenames with extensions
    body = re.sub(r'\b\w+\.\w+\b', '', body)
    # Remove any special characters, numbers, punctuation but retain alphabetic characters and spaces
    clean_text = re.sub(r'[^a-zA-Z\s]', ' ', body)

    # Normalize whitespace
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text


In [113]:
msg = emails.sample().iloc[0]['message']

display(clean_email_text(msg))
print(msg)

'AMEREX Diana Sean I am missing the following deal Enron sells to Powerex MId C light at Chris Mallory Amerex does not recognize deal BLOOMBERG All deals are PREBON Chris Mallory deal Is this a duplicate of because Prebon only recognizes deal deal Prebon says should be heavy not light hrs'

Message-ID: <24659304.1075841742070.JavaMail.evans@thyme>
Date: Wed, 28 Mar 2001 08:32:00 -0800 (PST)
From: evelyn.metoyer@enron.com
To: kate.symes@enron.com
Subject: 3/38 Checkout
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Evelyn Metoyer
X-To: Kate Symes
X-cc: 
X-bcc: 
X-Folder: \kate symes 6-27-02\Notes Folders\Deal communication\Deal discrepancies
X-Origin: SYMES-K
X-FileName: kate symes 6-27-02.nsf

AMEREX

Diana/Sean
I am missing the following deal:
1) Enron sells to Powerex MId-C light 3/29 at $140.00


Chris Mallory
Amerex does not recognize deal 563362




BLOOMBERG
All deals are o.k.





PREBON

Chris Mallory
deal 563461
Is this a duplicate of 563460 because Prebon only recognizes 1 deal

deal 563358
Prebon says should be heavy not light hrs.
















In [5]:
emails['cleaned_message'] = emails['message'].apply(clean_email_text)

In [6]:
pd.set_option('display.max_colwidth', 10000)
emails.sample(5)

file  \
717          allen-p/all_documents/204.   
87706                  dean-c/inbox/90.   
333635  mclaughlin-e/deleted_items/304.   
237586          kean-s/attachments/207.   
144718              grigsby-m/inbox/15.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [8]:
# emails.dropna().drop(columns=['message']).to_csv('cleaned_msgs.csv')

import json

filtered = emails.dropna().drop(columns=['message'])
filtered = filtered[filtered['cleaned_message'] != '']
filtered

with open('corpus.json', 'w') as file:
        json.dump(filtered['cleaned_message'].str.lower().str.split(' ').tolist(), file)

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [1]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False


class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        for word in vocabulary:
            self.insert(word)

    def insert(self, word: str):
        """
        Inserts a word into the prefix tree.
        """
        current_node = self.root
        for char in word:
            # If character not in current node's children, add it
            if char not in current_node.children:
                current_node.children[char] = PrefixTreeNode()
            # Move to the next node
            current_node = current_node.children[char]
        # Mark the end of a word
        current_node.is_end_of_word = True

    def search_prefix(self, prefix: str) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        result = []
        current_node = self.root

        for char in prefix:
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                return result

        def collect_all_words(node, path):
            if node.is_end_of_word:
                result.append(path)
            for char, child_node in node.children.items():
                collect_all_words(child_node, path + char)

        # Collect all words starting from the end of the prefix
        collect_all_words(current_node, prefix)

        return result

In [36]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])
print('success')


success


Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [3]:
from typing import Tuple
from collections import Counter


class WordCompletor:
    def __init__(self, corpus: List[str]):
        """
        corpus: list – корпус текстов
        """
        counter = Counter()
        for words in corpus:
            counter.update(words)

        total = sum(counter.values())
        self.probs = {w: p/total for w, p in counter.items()}
        self.vocabulary = counter.keys()

        self.prefix_tree = PrefixTree(self.vocabulary)

    def get_words_and_probs(self, prefix: str) -> Tuple[List[str], List[float]]:
        # Find all words starting with the given prefix
        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.probs[word] for word in words]
        return words, probs

In [35]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}
print('success')


success


## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [27]:
from collections import defaultdict


class NGramLanguageModel:
    def __init__(self, corpus, n):
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()
        self.n = n

        for words in corpus:
            for i in range(len(words)):
                for j in range(i+1, min(len(words), i+n+1)):
                    context = tuple(words[i:j])
                    word = words[j]
                    self.ngram_counts[context][word] += 1
                    self.context_counts[context] += 1



    def get_next_words_and_probs(self, prefix: list) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """
        context = tuple(prefix[-self.n:])
        next_words_counter = self.ngram_counts.get(context, Counter())

        if not next_words_counter:
            return [], []

        total_count = self.context_counts[context]
        next_words = list(next_words_counter.keys())
        probs = [count / total_count for count in next_words_counter.values()]

        return next_words, probs


In [34]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n = 2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}
print('success')


success


Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [31]:
from typing import Union
import heapq


def get_top_n_words(words: List[str], probs: List[float], n: int) -> List[str]:
    top_n = heapq.nlargest(n, zip(probs, words))
    top_n_words = [word for _, word in top_n]
    return top_n_words


def get_top_word(words: List[str], probs: List[float]) -> str:
    max_p, max_w = 0, None
    for w, p in zip(words, probs):
        if p > max_p:
            max_p, max_w = p, w
    return max_w


class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, need_correction=True, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        need_correction: нужно ли дополнять последнее слово
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        if isinstance(text, str):
            text = text.split()

        if not text:
            return []

        suggestions = []
        last_word = text[-1]
        extended_text = text
        if need_correction:
            completion = get_top_word(*self.word_completor.get_words_and_probs(last_word))
            if completion:
                extended_text = text[:-1] + [completion]
                last_word = completion

        next_words = self.n_gram_model.get_next_words_and_probs(extended_text)
        next_words = get_top_n_words(*next_words, n_texts)
        if next_words == []:
            return []

        for n_w in next_words:
            new_ext_text = extended_text + [n_w]
            current_suggestion = [last_word, n_w]
            for _ in range(n_words-1):
                words = self.n_gram_model.get_next_words_and_probs(new_ext_text)

                next_word = get_top_word(*words)
                current_suggestion.append(next_word)
                extended_text = extended_text[1:] + [next_word]
            suggestions.append(current_suggestion)

        return suggestions

In [33]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)



assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]
print('success')

success


In [23]:
text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1)

new_ext_text:  ['aa', 'aa', 'aa'] ['aa', 'aa']
words:  ([], [])
words:  ([], [])


[['aa', 'aa', None, None]]

In [39]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!